In [99]:
from dotenv import load_dotenv  
import os  
# Configure logger
import logging
import sys
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import CodeInterpreterTool
from azure.ai.agents.models import ListSortOrder
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval, MessageTextContent

# Load the .env file  
load_dotenv(override=True) 

# Now you can use the environment variables, for example:  
project_endpoint = os.environ["PROJECT_ENDPOINT"]
model=os.environ["MODEL_DEPLOYMENT_NAME"]


In [81]:
# Retrieve the project endpoint from environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]


# Initialize the AIProjectClient
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

In [82]:

agent_name = os.environ["AZURE_MCP_AGENT_NAME"]
agent_list = project_client.agents.list_agents()

agent = None
if agent_list:
    for agent_object in agent_list:
        if agent_object.name == agent_name:
            agent = agent_object

if not agent:
    print(f"Agent '{agent_name}' not found in the project.")
else:
    print(f"Agent '{agent_name}' found in the project.")
    print(f"Agent ID: {agent.id}")

Agent 'mmx-ai-mcp-agent' found in the project.
Agent ID: asst_Xo8NrkiCzJRNT9bTGsniN8R1


In [83]:
mcp_server_url = "https://gitmcp.io/Azure/azure-rest-api-specs"
mcp_server_label = "rest_api_specs"

project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
    #exclude_environment_credential=True,
    #exclude_managed_identity_credential=True
)

agents_client = project_client.agents

In [84]:
if not agent:
    print(f"Agent with name '{agent_name}' not found, creating a new agent.")

    # Initialize agent MCP tool
    mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
    )

    print(f"Allowed tools: {mcp_tool.allowed_tools}")

    # Create a new agent with the mcp tool definitions
    agent = agents_client.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name=agent_name,
        instructions="""
        You are a helpful assistant. Use the tools provided to answer the user's questions.  Be sure to cite your sources.""",
        tools=mcp_tool.definitions,
)
    print(f"Created agent, ID: {agent.id}")
else:
    print(f"Found agent by name '{agent_name}', ID={agent.id}")



Found agent by name 'mmx-ai-mcp-agent', ID=asst_Xo8NrkiCzJRNT9bTGsniN8R1


In [85]:
thread = agents_client.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_AIdOuBMSUKuUq1RFmCdiDf3X


In [86]:
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
    TEll me about the Azure REST API specs
    """
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_DtFZLH5xKrVyUfqUFRYnba5M


In [87]:
run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
print(f"Created run, ID: {run.id}")

Created run, ID: run_3g6IRalWzoXyyg7b5IO5OX72


In [88]:
import time

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    print(f"Waiting for run to complete, current status: {run.status}")
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
        tool_calls = run.required_action.submit_tool_approval.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
            break

        tool_approvals = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredMcpToolCall):
                try:
                    print(f"Approving tool call: {tool_call}")
                    tool_approvals.append(
                        ToolApproval(
                            tool_call_id=tool_call.id,
                            approve=True,
                            headers=mcp_tool.headers,
                        )
                    )
                except Exception as e:
                    print(f"Error approving tool_call {tool_call.id}: {e}")

        print(f"tool_approvals: {tool_approvals}")
        if tool_approvals:
            agents_client.runs.submit_tool_outputs(
                thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
            )

print(f"Current run status: {run.status}")


Waiting for run to complete, current status: RunStatus.QUEUED
Waiting for run to complete, current status: RunStatus.IN_PROGRESS
Waiting for run to complete, current status: RunStatus.IN_PROGRESS
Waiting for run to complete, current status: RunStatus.IN_PROGRESS
Waiting for run to complete, current status: RunStatus.IN_PROGRESS
Waiting for run to complete, current status: RunStatus.IN_PROGRESS
Waiting for run to complete, current status: RunStatus.IN_PROGRESS
Waiting for run to complete, current status: RunStatus.IN_PROGRESS
Waiting for run to complete, current status: RunStatus.IN_PROGRESS
Waiting for run to complete, current status: RunStatus.IN_PROGRESS
Waiting for run to complete, current status: RunStatus.IN_PROGRESS
Waiting for run to complete, current status: RunStatus.IN_PROGRESS
Waiting for run to complete, current status: RunStatus.IN_PROGRESS
Waiting for run to complete, current status: RunStatus.IN_PROGRESS
Waiting for run to complete, current status: RunStatus.IN_PROGRESS


In [89]:
if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    
# Fetch and log all messages
messages = project_client.agents.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Role: {message.role}, Content: {message.content}")

Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': "The Azure REST API specs refer to the formal OpenAPI (formerly known as Swagger) specifications used by Microsoft Azure to describe the REST APIs provided by their cloud platform. These specs are publicly available and maintained in the [Azure/azure-rest-api-specs GitHub repository](https://github.com/Azure/azure-rest-api-specs). Here’s an overview:\n\n## What Are Azure REST API Specs?\n\n- **OpenAPI Format**: The specs use the OpenAPI standard (YAML/JSON) to define endpoints, parameters, authentication, request/response formats, and more.\n- **Coverage**: Specs are provided for most Azure services (e.g., Compute, Storage, Networking, Key Vault, Cosmos DB).\n- **Purpose**: They enable developers, SDK generators, tools, and integrations to interact with Azure services programmatically and in a standardized way.\n\n## Key Features\n\n- **Comprehensive**: Covers almost all management (ARM) and many data-plane (service-

In [96]:
import json
from IPython.display import display, Markdown

run_steps = project_client.agents.run_steps.list(thread_id = thread.id, run_id = run.id)

for step in run_steps:
    print(f"Step ID: {step.id}, Status: {step.status}, type: {step.type}")
    if step.type == "tool_calls":
        print(f"Too call details:")
        for tool_call in step.step_details.tool_calls:
            print(json.dumps(tool_call.as_dict(), indent = 2))


Step ID: step_0UBtkZp5xOLnPBvoOuP8OWdN, Status: RunStepStatus.COMPLETED, type: RunStepType.MESSAGE_CREATION


In [98]:
messages = project_client.agents.messages.list(thread_id = thread.id, run_id = run.id, order = ListSortOrder.ASCENDING)

for message in messages:
    last_message = message.content[-1]
    if isinstance(last_message, MessageTextContent):
        display(Markdown(f"{message.role}"))
        display(Markdown(f"{last_message.text.value}"))

MessageRole.AGENT

The Azure REST API specs refer to the formal OpenAPI (formerly known as Swagger) specifications used by Microsoft Azure to describe the REST APIs provided by their cloud platform. These specs are publicly available and maintained in the [Azure/azure-rest-api-specs GitHub repository](https://github.com/Azure/azure-rest-api-specs). Here’s an overview:

## What Are Azure REST API Specs?

- **OpenAPI Format**: The specs use the OpenAPI standard (YAML/JSON) to define endpoints, parameters, authentication, request/response formats, and more.
- **Coverage**: Specs are provided for most Azure services (e.g., Compute, Storage, Networking, Key Vault, Cosmos DB).
- **Purpose**: They enable developers, SDK generators, tools, and integrations to interact with Azure services programmatically and in a standardized way.

## Key Features

- **Comprehensive**: Covers almost all management (ARM) and many data-plane (service-level) APIs.
- **Enables SDK Generation**: Used to auto-generate Azure SDKs for different languages (Python, .NET, Java, Go, etc.).
- **Consistency**: Ensures API updates and new Azure services are documented in a consistent, discoverable way.

## Structure

- **GitHub Repository**: [Azure/azure-rest-api-specs](https://github.com/Azure/azure-rest-api-specs)
  - Organized by service (e.g., `specification/storage/resource-manager`)
  - Includes readme files, version folders, and OpenAPI (swagger) definitions

## Usage

- **Developers**: Reference specs for direct REST API integration or to understand Azure’s REST contract.
- **Tooling**: Used by tools like [Azure REST API Reference Docs](https://learn.microsoft.com/en-us/rest/api/azure/) and SDK generators.
- **Validation & Linting**: Spec files are validated and linted as part of Microsoft’s API lifecycle.

## Example

Here’s a brief excerpt of an OpenAPI spec for a storage account list operation (YAML):

```yaml
paths:
  /subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/providers/Microsoft.Storage/storageAccounts:
    get:
      summary: Lists all storage accounts
      parameters: [...]
      responses:
        '200':
          description: Successful operation
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/StorageAccountListResult'
```

## Resources

- [Azure REST API Reference Docs](https://learn.microsoft.com/en-us/rest/api/azure/)
- [Azure/azure-rest-api-specs GitHub Repository](https://github.com/Azure/azure-rest-api-specs)
- [OpenAPI Specification](https://swagger.io/specification/)

Let me know if you want details about a specific service or API!